In [2]:
from env import KEY
import requests
import pandas as pd
import metadata_parser
import json


def airtable_request(url_base):

    headers = {"Authorization": f"Bearer {KEY}"}
    offset = 0
    record_list = []

    while offset is not None:
        if offset != 0:
            url = url_base + "?offset=" + offset
        else:
            url = url_base

        response = requests.request("GET", url, headers=headers)

        responses = response.json()
        r = [r for r in responses["records"]]
        record_list += r
        offset = responses.get("offset")

    return record_list


In [7]:
page.metadata['og']

{'type': 'article',
 'title': 'The Most Diverse Cities Are Often The Most Segregated',
 'url': 'https://fivethirtyeight.com/features/the-most-diverse-cities-are-often-the-most-segregated/',
 'description': 'When I was a freshman at the University of Chicago in 1996, I heard the same thing again and again: Do not leave the boundaries of Hyde Park. Do not go north of…',
 'site_name': 'FiveThirtyEight',
 'image': 'https://fivethirtyeight.com/wp-content/uploads/2015/04/461256345-e1430436547244.jpg?w=712',
 'image:width': '712',
 'image:height': '534',
 'image:alt': '',
 'locale': 'en_US'}

In [6]:
page.metadata['og']['image']

/Users/austinsteinhart/anaconda3/lib/python3.10/site-packages/metadata_parser/__init__.py:1619: FutureWarning: MetadataParser.metadata is deprecated in 1.0; Operate on the parsed result directly.
  warn_future(


'https://fivethirtyeight.com/wp-content/uploads/2015/04/461256345-e1430436547244.jpg?w=712'

In [2]:
url = 'https://www.nytimes.com/interactive/2019/04/08/upshot/democratic-electorate-twitter-real-life.html'
page = metadata_parser.MetadataParser(url=url)
page.metadata['og']

/Users/austinsteinhart/anaconda3/lib/python3.10/site-packages/metadata_parser/__init__.py:1543: FutureWarning: `search_head_only` was not provided and defaulting to `True` Future versions will default to `False`.
  warn_future(
MetadataParser.fetch_url(https://www.nytimes.com/interactive/2019/04/08/upshot/democratic-electorate-twitter-real-life.html)
/Users/austinsteinhart/anaconda3/lib/python3.10/site-packages/metadata_parser/__init__.py:1619: FutureWarning: MetadataParser.metadata is deprecated in 1.0; Operate on the parsed result directly.
  warn_future(


{'url': 'https://www.nytimes.com/interactive/2019/04/08/upshot/democratic-electorate-twitter-real-life.html',
 'type': 'article',
 'title': 'The Democratic Electorate on Twitter Is Not the Actual Democratic Electorate (Published 2019)',
 'image': 'https://static01.nyt.com/images/2019/04/08/upshot/democratic-electorate-twitter-real-life-1554759902835/democratic-electorate-twitter-real-life-1554759902835-facebookJumbo-v4.jpg?year=2019&h=549&w=1050&s=d366f395b04658ce13725cf0e4de60af17a742bc140e2ddf7a77369d82fcd750&k=ZQJBKqZ0VN',
 'image:alt': '',
 'description': 'A detailed look at the voters with the numbers to decide the 2020 Democratic nominee.'}

In [3]:
def clean_records():

    # pull in data and format
    article_url = "https://api.airtable.com/v0/appfRI9pJ5OIOt9LV/for_api"
    article_list = airtable_request(article_url)
    art_df = pd.DataFrame([r["fields"] for r in article_list])

    tab_url = "https://api.airtable.com/v0/appfRI9pJ5OIOt9LV/the_tab"
    tab_list = airtable_request(tab_url)
    tab_df = pd.DataFrame([r["fields"] for r in tab_list])

    pubs_url = "https://api.airtable.com/v0/appfRI9pJ5OIOt9LV/Publications"
    pub_list = airtable_request(pubs_url)
    pub_list_clean = {r["id"]:r["fields"]["Name"] for r in pub_list}

    art_df['tags'] = art_df['tags'].fillna("").apply(list)
    art_df['topics'] = art_df['topic'] + art_df['tags']
    art_df["type"] = "Article"
    art_df["pub"] = art_df["pub"].str[0].map(pub_list_clean)

    art_clean = art_df[["title", "pub", "link", "topics","type", "pub_date"]]

    tab_df["pub"] = ""
    tab_df["topics"] = tab_df["topic"]
    tab_clean = tab_df[["title", "pub", "link", "topics", "type","pub_date"]]   

    records = pd.concat([art_clean,tab_clean]).reset_index(drop=True)
    
    return records


In [38]:
records = clean_records()

In [5]:
def get_meta(row):
    meta = []
    try:
        print(f"trying row {row['title']}")
        page = metadata_parser.MetadataParser(url=row["link"])
        meta.append(page.metadata['og'])
        print(f"sucess row {row['title']}")
    except:
        print(f"oopsies on row {row['title']}")

    return meta


def add_image(row, clean_meta_dict):
    for r in clean_meta_dict:
        if r.get("image") and r.get("url"):
            if r["url"] == row["link"].split("?")[0]:
                return r["image"]
            

def add_description(row, clean_meta_dict):
    for r in clean_meta_dict:
        if r.get("description") and r.get("url"):
            if r["url"] == row["link"].split("?")[0]:
                return r["description"]
    

In [ ]:
records_meta = records.drop([210,241])
meta  = records_meta.apply(get_meta, axis=1)
clean_meta_dict = [r[0] for r in meta.tolist() if len(r)>0 and len(r[0])>0]


In [6]:
def scrape_metadata(export=False):
    records_meta = records.drop([210,241])
    meta  = records_meta.apply(get_meta, axis=1)
    clean_meta_dict = [r[0] for r in meta.tolist() if len(r)>0 and len(r[0])>0]
    if export: 
        with open('clean_meta.json', 'w') as fp:
            json.dump(clean_meta_dict, fp)

In [ ]:
#scrape_metadata(True)

In [39]:
records["image"] = records.apply(add_image, args=(clean_meta,), axis=1)
records["description"] = records.apply(add_description, args=(clean_meta,), axis=1)

In [40]:
pd.set_option('display.max_rows', 100)

records


,title,pub,link,topics,type,pub_date,image,description
0,"Steady Jobs, With Pay and Hours That Are Anyth...",The New York Times,https://www.nytimes.com/2017/05/31/business/ec...,"[Economics, Visualization]",Article,2017-05-31,None,None
1,The Unmet Promise of Equality,The New York Times,https://www.nytimes.com/interactive/2018/02/28...,"[Economics, Visualization]",Article,2018-02-28,https://static01.nyt.com/images/2018/03/01/opi...,"Still much to heal in a divided society, a hal..."
2,America's Most and Least Distressed Cities,City Lab,https://www.citylab.com/equity/2017/09/distres...,"[Stockton, Economics]",Article,2017-09-26,None,None
3,Existential Office Emails,The New Yorker,https://www.newyorker.com/humor/daily-shouts/e...,[Life and Love],Article,2017-03-06,https://media.newyorker.com/photos/59097ef9c14...,"“Morning, Rob. Hope your weekend was an effect..."
4,Record Wildfires on the West Coast Are Capping...,The New York Times,https://www.nytimes.com/interactive/2020/09/24...,"[Science and Technology, Visualization]",Article,2020-09-24,https://static01.nyt.com/images/2020/09/23/us/...,"One after another, major fires exploded across..."
...,...,...,...,...,...,...,...,...
357,8 Reasons Self-Care Is So Hard,,https://medium.com/n-q-v/9-reasons-self-care-i...,Life and Love,Other,2019-07-14,None,None
358,The Faces That Look Back at Us When We Come Ou...,,https://www.nytimes.com/interactive/2022/09/15...,Culture,Article,2022-09-15,https://static01.nyt.com/images/2022/09/14/art...,"On television, for over 50 years, queer charac..."
359,Luis Miguel: La Serie,,https://www.netflix.com/title/80191236,Culture,TV Show / Movie,2018-04-18,https://occ-0-92-7.1.nflxso.net/dnm/api/v6/E8v...,This series dramatizes the life story of Mexic...
360,Why is the Wartime Press Corps so Hawkish?,,https://foreignpolicy.com/2022/03/30/ukraine-w...,Politics,Article,2022-03-30,https://foreignpolicy.com/wp-content/uploads/2...,The United States’ most reputable media outlet...


In [42]:
import numpy as np
records["is_image"] = np.where(records["image"].notna(), True, False)
records.sort_values(by=["is_image", "pub_date"], ascending=False)

,title,pub,link,topics,type,pub_date,image,description,is_image
90,The USDA’s gardening zones shifted.,NPR,https://apps.npr.org/plant-hardiness-garden-map/,"[Visualization, Science and Technology]",Article,2024-05-13,https://apps.npr.org/plant-hardiness-garden-ma...,There's a good chance your zone shifted when t...,True
149,"10 Years, 100 Stories: The Work That Defines t...",The Upshot,https://www.nytimes.com/interactive/2024/04/27...,[Visualization],Article,2024-04-27,https://static01.nyt.com/images/2024/05/24/rea...,The Upshot is 10 years old this week. Here’s a...,True
108,"Mapping America’s access to nature, neighborho...",The Washington Post,https://www.washingtonpost.com/climate-environ...,"[Visualization, Science and Technology]",Article,2024-04-10,https://www.washingtonpost.com/wp-apps/imrs.ph...,"Spending time in nature is linked to a longer,...",True
53,We Tried to Create a Diverse College Class Wit...,The New York Times,https://www.nytimes.com/interactive/2024/03/09...,"[Education, Visualization]",Article,2024-03-09,https://static01.nyt.com/images/2024/03/08/mul...,The Supreme Court effectively ended race-based...,True
30,10 Data Points and Documents That Made Us 🤔 in...,The Upshot,https://www.nytimes.com/interactive/2023/12/30...,[Visualization],Article,2023-12-30,https://static01.nyt.com/images/2023/12/29/mul...,The Upshot staff shares a selection of their f...,True
...,...,...,...,...,...,...,...,...,...
88,The Depressed Person,David Foster Wallace,https://harpers.org/wp-content/uploads/Harpers...,[Life and Love],Article,1998-01-01,None,None,False
354,The Elephant Man,,http://azactorsacademy.com/uploads/plays/eleph...,Life and Love,Other,1977-11-07,None,None,False
85,A Letter to My Nephew,NaN,http://progressive.org/magazine/letter-nephew/,[Social Issues],Article,1962-12-01,None,None,False
95,How to Build a Successful Team,The New York Times,https://www.nytimes.com/guides/business/manage...,[Life and Love],Article,NaN,None,None,False


In [48]:
records['image'] = records['image'].fillna("")
records[records['image'].str.contains("ffrock-1470254958-36.jpg").fillna(False)]


,title,pub,link,topics,type,pub_date,image,description,is_image
155,The Ballad of Rocky Rontal: How Do We Forgive ...,The California Sunday Magazine,https://story.californiasunday.com/rocky-rontal,"[Stockton, Life and Love]",Article,2016-08-04,scale~1200x1200~0807ffrock-1470254958-36.jpg,Stockton's street wars sent young lives off tr...,True


In [53]:
records.iloc[155].image = ""

/var/folders/vk/_03tl7qs1sv80_pgkhd8bxch0000gn/T/ipykernel_15467/235975971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records.iloc[155].image = ""


In [59]:
records["image"] = np.where(records["image"] == "scale~1200x1200~0807ffrock-1470254958-36.jpg", "", records["image"])

In [60]:
records.loc[155]

title          The Ballad of Rocky Rontal: How Do We Forgive ...
pub                               The California Sunday Magazine
link             https://story.californiasunday.com/rocky-rontal
topics                                 [Stockton, Life and Love]
type                                                     Article
pub_date                                              2016-08-04
image                                                           
description    Stockton's street wars sent young lives off tr...
is_image                                                    True
Name: 155, dtype: object